In [1]:
import os
from openai import OpenAI



In [2]:
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key='sk-4a47da58c2e64a53bc7b94d0892016be',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    model="qwen-plus",  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你几岁了？？'}
        ]
)
print(completion.choices[0].message.content)

我是一个AI助手，没有实际年龄。我很高兴能与您交流，为您提供帮助。


In [3]:

def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

client = OpenAI(
    api_key="sk-fd20bf9c2c0f4c9ea49ae5ed53037504",
    base_url="https://api.deepseek.com",
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages)
print(f"User>\t {messages[0]['content']}")

tool = message.tool_calls[0]
messages.append(message)

messages.append({"role": "tool", "tool_call_id": tool.id, "content": "24℃"})
message = send_messages(messages)
print(f"Model>\t {message.content}")

User>	 How's the weather in Hangzhou?
Model>	 The current weather in Hangzhou is 24°C.


In [4]:
messages

[{'role': 'user', 'content': "How's the weather in Hangzhou?"},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_5b2423d7-8642-4808-aed4-ed6accc87ead', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)]),
 {'role': 'tool',
  'tool_call_id': 'call_0_5b2423d7-8642-4808-aed4-ed6accc87ead',
  'content': '24℃'}]

In [7]:
messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )

In [9]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_426f7148-722f-4722-ac83-c8f70d939a9f', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)]))

In [13]:
import requests
from PIL import Image
import io
def upload_image_to_imgbb(image_path: str, max_size: int = 512) -> str:
    # Open the image and resize it
    image = Image.open(image_path)
    
    # Resize image if needed (preserve aspect ratio)
    width, height = image.size
    if width > max_size or height > max_size:
        if width > height:
            new_width = max_size
            new_height = int(height * (max_size / width))
        else:
            new_height = max_size
            new_width = int(width * (max_size / height))
        image = image.resize((new_width, new_height), Image.LANCZOS)
    
    # Save to buffer
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)
    
    # Upload the resized image
    response = requests.post(
        "https://api.imgbb.com/1/upload",
        params={"key": '9689e42764a7f935277d8371a415dc4d'},
        files={"image": buffer}
    )
    data = response.json()
    return data["data"]["url"]

In [14]:
url  = upload_image_to_imgbb("/Users/yingcongchen/Downloads/Picture1.png")

In [16]:
from openai import OpenAI
from dotenv import load_dotenv
import os


load_dotenv()



image_url = upload_image_to_imgbb("/Users/yingcongchen/Downloads/近照.jpg")
print("--------------------------------")
print(image_url)
print("--------------------------------")

client = OpenAI(
    api_key=os.getenv('QWEN_API_KEY'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-vl-plus",  # 此处以qwen-vl-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "请分析这张图像的内容"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://i.ibb.co/bRXX1Yz6/image.png"
                    }
                }
            ]
        }
    ]
    )


--------------------------------
https://i.ibb.co/8LGxq0Bx/image.png
--------------------------------


BadRequestError: Error code: 400 - {'error': {'code': 'InvalidParameter.DataInspection', 'param': None, 'message': 'Download the media resource timed out during the data inspection process.', 'type': 'InvalidParameter.DataInspection'}, 'id': 'chatcmpl-0983b101-9628-928c-8cd2-cf2848f4764d', 'request_id': '0983b101-9628-928c-8cd2-cf2848f4764d'}

In [30]:
from typing import List, Dict, Any, Optional
import json
def format_tools_description(tools: List[Dict[str, Any]]) -> str:
    """
    Format tool descriptions for inclusion in prompts.
    
    Args:
        tools: List of available tools
        
    Returns:
        Formatted string describing available tools
    """
    descriptions = []
    
    # Add a concise guide for tool calling at the beginning
    format_explanation = """ Note that only the following tools are available:
"""
    descriptions.append(format_explanation)
    
    # Generate description for each tool
    for tool in tools:
        name = tool.get("function", {}).get("name")
        description = tool.get("function", {}).get("description")
        params = tool.get("function", {}).get("parameters", {}).get("properties", {})
        required_params = tool.get("function", {}).get("parameters", {}).get("required", [])
        
        if not name or not description:
            continue
            
        # Format parameter descriptions
        param_desc = []
        for param_name, param_info in params.items():
            param_type = param_info.get("type", "any")
            param_description = param_info.get("description", "")
            required = "required" if param_name in required_params else "optional"
            param_desc.append(f"  - {param_name} ({param_type}, {required}): {param_description}")
            
        param_text = "\n".join(param_desc) if param_desc else "  No parameters"
        
        # # Create a simple example with actual parameter names
        # example_args = {}
        # for param_name in params:
        #     if param_name in required_params:
        #         if params[param_name].get("type") == "number" or params[param_name].get("type") == "integer":
        #             example_args[param_name] = 0
        #         else:
        #             example_args[param_name] = f"<{param_name}>"
        
        # example = f"""Example: {{"name": "{name}", "arguments": {json.dumps(example_args)}}}"""
            
        # Combine all into the tool description
        descriptions.append(f"Tool: {name}\nDescription: {description}\nParameters:\n{param_text}")
        
    return "\n\n".join(descriptions)

In [28]:
def _format_tools_description(tools: List[Dict[str, Any]]) -> str:
    """
    Format tool descriptions for inclusion in prompts.
    
    Args:
        tools: List of available tools
        
    Returns:
        Formatted string describing available tools
    """
    descriptions = []
    for tool in tools:
        name = tool.get("function", {}).get("name")
        description = tool.get("function", {}).get("description")
        params = tool.get("function", {}).get("parameters", {}).get("properties", {})
        
        if not name or not description:
            continue
            
        param_desc = []
        for param_name, param_info in params.items():
            param_type = param_info.get("type", "any")
            param_description = param_info.get("description", "")
            param_desc.append(f"  - {param_name} ({param_type}): {param_description}")
            
        param_text = "\n".join(param_desc) if param_desc else "  No parameters"
        descriptions.append(f"- {name}: {description}\n  Parameters:\n{param_text}")
    return "\n\n".join(descriptions)

In [29]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of a location, the user should supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_restaurants",
            "description": "Search for restaurants in a specific area with optional cuisine type",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city or area to search in"
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "Type of cuisine (e.g., Italian, Chinese, etc.)"
                    },
                    "price_range": {
                        "type": "integer",
                        "description": "Price range from 1 (cheap) to 4 (expensive)"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

print(format_tools_description(tools))

- get_weather: Get weather of a location, the user should supply a location first
  Parameters:
  - location (string): The city and state, e.g. San Francisco, CA

- search_restaurants: Search for restaurants in a specific area with optional cuisine type
  Parameters:
  - location (string): The city or area to search in
  - cuisine (string): Type of cuisine (e.g., Italian, Chinese, etc.)
  - price_range (integer): Price range from 1 (cheap) to 4 (expensive)
